In [1]:
import os
import numpy as np

import matplotlib.pyplot as plt
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
import glob
import pickle
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
from keras.layers import LeakyReLU
from keras.optimizers import Adam, Nadam
import keras
import random 

In [3]:
!ls

cnn_proj_yvg_testing          librit_1sec_hdpickle
final_cnn.ipynb               librit_1sec_hdpickle.zip
final_cnn_final_pickles.ipynb repickle_all_final.py
librispeech_1sec_hdpickle     spectrogram_svm.ipynb
librispeech_1sec_hdpickle.zip


In [4]:
DATA_PATH = "cnn_proj_yvg_testing"

with open(os.path.join(DATA_PATH, "val_1_sec_hd_feature.pickle"), 'rb') as jar:
    x_val = pickle.load(jar)

with open(os.path.join(DATA_PATH, "val_1_sec_hd_label.pickle"), 'rb') as jar:
    y_val = pickle.load(jar)

print(y_val.shape)

(4196,)


In [5]:
def imNorm(arr):
    for i in range(arr.shape[0]):

        arr[i] -= np.min(arr[i])
        arr[i] = np.minimum(arr[i], 255)
    return arr

In [12]:
file_path = "cnn_proj_yvg_testing/subPickles"

for i in range(1):
    # load features 
    x_name = 'x_train_splitted' + str(i) + '.pickle'
    file_name = os.path.join(file_path, x_name) 
    print("training on pickle", x_name)
    with open(file_name, 'rb') as jar:
        x_train = pickle.load(jar)
    x_train = imNorm(x_train)
    if i == 0:
        x_train_all = x_train[:1000]
    else:
        x_train_all = np.concatenate((x_train_all,x_train),axis = 0)
    print(x_train_all.shape)
    
    y_name = 'y_train_splitted' + str(i) + '.pickle'
    file_name = os.path.join(file_path, y_name)
    with open(file_name, 'rb') as jar:
        y_train = pickle.load(jar)
    
    if i == 0:
        y_train_all = y_train[:1000]
    else:
        y_train_all = np.concatenate((y_train_all,y_train),axis = 0)
    print(y_train_all.shape)

training on pickle x_train_splitted0.pickle
(1000, 201, 66, 1)
(1000,)


In [13]:
x_train_svm = x_train_all.reshape(x_train_all.shape[0],x_train_all.shape[1]*x_train_all.shape[2] )
x_val_svm = x_val.reshape(x_val.shape[0],x_val.shape[1]*x_val.shape[2] )


In [14]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(gamma = 'auto',verbose = 1)
clf.fit(x_train_svm, y_train_all)
print("train", clf.score(x_train_svm, y_train_all))


[LibSVM]train 1.0


In [15]:
y_pred = clf.predict(x_val_svm)
score = accuracy_score(y_val, y_pred)
print("val ", score)

val  0.5367016205910391


In [17]:
from sklearn.dummy import DummyClassifier

uniform = DummyClassifier(strategy='most_frequent', random_state=2)
uniform.fit(x_train_svm, y_train)

print(uniform.score(x_train_svm, y_train))

print(uniform.score(x_val_svm, y_val))

ValueError: Found input variables with inconsistent numbers of samples: [7551, 1000]

In [18]:
len(y_val)


4196

In [20]:
print(*zip(y_pred,y_val))

(1.0, 0.0) (1.0, 0.0) (1.0, 0.0) (1.0, 1.0) (1.0, 1.0) (1.0, 1.0) (1.0, 1.0) (1.0, 0.0) (1.0, 1.0) (1.0, 1.0) (1.0, 1.0) (1.0, 0.0) (1.0, 0.0) (1.0, 1.0) (1.0, 1.0) (1.0, 0.0) (1.0, 1.0) (1.0, 1.0) (1.0, 1.0) (1.0, 1.0) (1.0, 1.0) (1.0, 0.0) (1.0, 0.0) (1.0, 1.0) (1.0, 1.0) (1.0, 1.0) (1.0, 1.0) (1.0, 1.0) (1.0, 0.0) (1.0, 1.0) (1.0, 1.0) (1.0, 1.0) (1.0, 0.0) (1.0, 1.0) (1.0, 0.0) (1.0, 0.0) (1.0, 0.0) (1.0, 0.0) (1.0, 0.0) (1.0, 1.0) (1.0, 0.0) (1.0, 0.0) (1.0, 0.0) (1.0, 0.0) (1.0, 0.0) (1.0, 0.0) (1.0, 0.0) (1.0, 1.0) (1.0, 1.0) (1.0, 1.0) (1.0, 0.0) (1.0, 1.0) (1.0, 1.0) (1.0, 1.0) (1.0, 0.0) (1.0, 0.0) (1.0, 0.0) (1.0, 0.0) (1.0, 0.0) (1.0, 1.0) (1.0, 0.0) (1.0, 1.0) (1.0, 0.0) (1.0, 0.0) (1.0, 0.0) (1.0, 0.0) (1.0, 0.0) (1.0, 1.0) (1.0, 0.0) (1.0, 0.0) (1.0, 1.0) (1.0, 1.0) (1.0, 1.0) (1.0, 1.0) (1.0, 1.0) (1.0, 1.0) (1.0, 0.0) (1.0, 1.0) (1.0, 1.0) (1.0, 0.0) (1.0, 1.0) (1.0, 1.0) (1.0, 0.0) (1.0, 0.0) (1.0, 1.0) (1.0, 1.0) (1.0, 0.0) (1.0, 1.0) (1.0, 1.0) (1.0, 1.0) (1.0, 0.0)